# Moving window operations

Objective: apply a moving window algorithm along the lat and lon axes by applying a kernel function on the time axis.

In [1]:
println(versioninfo())
using Pkg; Pkg.activate("."); Pkg.instantiate()
using Zarr, EarthDataLab, YAXArrays, Statistics

Julia Version 1.9.1
Commit 147bdf428cd (2023-06-07 08:27 UTC)
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 10 × Apple M1 Pro
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, apple-m1)
  Threads: 20 on 8 virtual cores
Environment:
  JULIA_NUM_THREADS = 20
nothing


  Activating project at `~/BGI/DeepCube/DaCuMoWin`


In [2]:
#Create a dummy array
a = Array{Union{Float64,Missing}}(rand(40,20,10))
lon = RangeAxis("Lon",1:40)
lat = RangeAxis("Lat",1:20)
tim = RangeAxis("Time",1:10)
c = YAXArray([lon,lat,tim],a)

YAXArray with the following dimensions
Lon                 Axis with 40 Elements from 1 to 40
Lat                 Axis with 20 Elements from 1 to 20
Time                Axis with 10 Elements from 1 to 10
Total size: 62.5 KB


In [3]:
# access remote data cube

rc = Cube(open_dataset(zopen("https://storage.de.cloud.ovh.net/v1/AUTH_84d6da8e37fe4bb5aea18902da8c1170/uc3/UC3SubCube_ts.zarr", fill_as_missing=false)))


YAXArray with the following dimensions
x                   Axis with 1253 Elements from 18.70211622302644 to 28.899269063907617
y                   Axis with 983 Elements from 42.299194879353976 to 34.30110854569159
time                Axis with 4560 Elements from 2009-03-06T10:00:00 to 2021-08-29T10:00:00
Variable            Axis with 5 elements: avg_rh ignition_points burned_areas lst_day evi 
units: unitless
Total size: 104.62 GB


In [21]:
# subset cube
sc = subsetcube(rc, x = (20.9,21), y = (36.9,37), time = (Date(2021,1,1), Date(2021,1,3)), variable = "evi")

YAXArray with the following dimensions
x                   Axis with 13 Elements from 20.901182730245225 to 20.99891901945495
y                   Axis with 12 Elements from 36.99700118972647 to 36.90740959128422
time                Axis with 10 Elements from 2021-01-01T10:00:00 to 2021-01-10T10:00:00
units: unitless
Total size: 6.09 KB


The MovingWindowResults would come from a function which would compare all timeseries of neighbouring pixels in the moving window to the time series in the center of the window. Therefore, the compuation would depend on the lat, lon and time axis and would be looped across the Variable axis.

## Naive approach

In [5]:
function my_mapwindow(f, img, window)
    out = zeros(eltype(img), axes(img))
    R = CartesianIndices(img)
    I_first, I_last = first(R), last(R)
    Δ = CartesianIndex(ntuple(x->window[x] ÷ 2, ndims(img)))
    @inbounds @simd for I in R
        patch = max(I_first, I-Δ):min(I_last, I+Δ)
        out[I] = f(view(img, patch))
    end
    return out
end

my_mapwindow (generic function with 1 method)

In [6]:
@time my_mapwindow(mean, c.data, (3,3,1));

  0.130496 seconds (539.34 k allocations: 35.133 MiB, 8.91% gc time, 97.06% compilation time)


In [22]:
@time my_mapwindow(mean, sc.data, (3,3,1));

342.916705 seconds (5.02 M allocations: 3.044 GiB, 0.18% gc time, 0.16% compilation time)


## With YAXArrays functions mapCube and MovingWindow

`MovingWindow(desc, pre, after)`

Constructs a `MovingWindow` object to be passed to an InDims constructor to define that the axis in desc shall participate in the inner function (i.e. shall be looped over), but inside the inner function pre values before and after values after the center value will be passed as well.

For example passing `MovingWindow("Time", 2, 0)` will loop over the time axis and always pass the current time step plus the 2 previous steps. So in the inner function the array will have an additional dimension of size 3. 

In [8]:
# Define the input dimensions with time first (all time steps) and lon and lat each with one previous and one consecutive slice. 
indims = InDims( MovingWindow("Lon",1,1), MovingWindow("Lat",1,1))

@time r1 = mapCube(c, indims=indims, outdims=OutDims()) do xout,xin
    # Inside this function, xin will have size 1x3x3 (time x lon x lat)
    # xout should have size 1 (time)
    xout = mean(xin)
end

  8.293748 seconds (20.23 M allocations: 1.178 GiB, 4.07% gc time, 379.51% compilation time)


YAXArray with the following dimensions
Lon                 Axis with 40 Elements from 1 to 40
Lat                 Axis with 20 Elements from 1 to 20
Time                Axis with 10 Elements from 1 to 10
Total size: 62.5 KB


When applying a function on moving windows with mapCube, one should take care of susceptible missing values, as well as of what happens at the edges of the cube.

Adding the keyword argument `window_oob_value` to `InDims` : if one of the input dimensions is a MowingWindow, this value will be used to fill out-of-bounds areas

In [39]:
# Define the input dimensions with time first (all time steps) and lon and lat each with one previous and one consecutive slice. 
indims = InDims( "time", MovingWindow("x",1,1), MovingWindow("y",1,1), window_oob_value = -9999.0)

@time r1 = mapCube(sc, indims=indims, outdims=OutDims("time")) do xout,xin
    # Inside this function, xin will have size 10x3x3 (time x lon x lat)
    # xout should have size 10 (time)
    xout[:] = mapslices(x->mean(skipmissing(x)), xin; dims = (2,3))
end

Progress:  50%|████████████████████▌                    |  ETA: 0:00:01

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


  0.992324 seconds (724.31 k allocations: 47.176 MiB, 177.45% compilation time)


YAXArray with the following dimensions
time                Axis with 10 Elements from 2021-01-01T10:00:00 to 2021-01-10T10:00:00
x                   Axis with 13 Elements from 20.901182730245225 to 20.99891901945495
y                   Axis with 12 Elements from 36.99700118972647 to 36.90740959128422
Total size: 6.09 KB


In [25]:
sc20 = subsetcube(rc, x = (20.9,21), y = (36.9,37), time = Date(2020,1,1), variable = "evi")

YAXArray with the following dimensions
x                   Axis with 13 Elements from 20.901182730245225 to 20.99891901945495
y                   Axis with 12 Elements from 36.99700118972647 to 36.90740959128422
units: unitless
Total size: 624.0 bytes


In [26]:
@time my_mapwindow(mean, sc20.data, (3,3,1));

 32.170437 seconds (265.72 k allocations: 299.140 MiB, 0.55% gc time)


In [28]:
indims = InDims( MovingWindow("x",1,1), MovingWindow("y",1,1), window_oob_value = -9999.0)

@time r1 = mapCube(sc20, indims=indims, outdims=OutDims()) do xout,xin
    # Inside this function, xin will have size 1x3x3 (time x lon x lat)
    # xout should have size 1 (time)
    xout[] = mean(skipmissing(xin))
end

Progress:  50%|████████████████████▌                    |  ETA: 0:00:00

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


  0.751965 seconds (320.00 k allocations: 20.027 MiB, 62.91% compilation time)


YAXArray with the following dimensions
x                   Axis with 13 Elements from 20.901182730245225 to 20.99891901945495
y                   Axis with 12 Elements from 36.99700118972647 to 36.90740959128422
Total size: 624.0 bytes
